In [2]:
import numpy as np
import json
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"
plt.style.use('seaborn-deep')
plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
# import matplotlib.pyplot as plt
# from MSC import *
%config Completer.use_jedi = False
showOriginalModelString = True
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tellurium as te
from roadrunner import Config

# We do not want CONSERVED MOIETIES set to true in this case
Config.setValue(Config.LOADSBMLOPTIONS_CONSERVED_MOIETIES, False)

In [13]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import tellurium as te
# %config Completer.use_jedi = False
# showOriginalModelString = True
#TODO: watch out that ctrl gets reset at each calibration run
free_params = {
    'IL10_L': [0,1], # threshold of Low membership function
    'IL10_H': [10,100], # threshold of High membership function
    'early_diff_m': [0,1], # threshold of medium value for early diff
    'late_diff_m': [0,1], # threshold of medium value for late diff
    'maturity_t':[0,1], # early maturity threshold
    'diff_time':[15,30], # days required for full differentiation
    'ALP_maturity': [1,3], #TODO: correct this
    'ARS_maturity': [1,3] #TODO: correct this
}

class Fuzzy_MSC:
    def __init__(self,params):
        self.antecedents = []
        self.consequents = []
        self.params = params
        # TNFa_0 = self.params['TNFa'][0]; TNFa_1 = self.params['TNFa'][1]; TNFa_end = self.params['TNFa'][2] 
        IL10_0 = self.params['IL10'][0]; IL10_end = self.params['IL10'][-1]; 
        #// define antecedents
        # TNFa = ctrl.Antecedent(np.arange(TNFa_0, TNFa_end, .05), 'TNFa')
        IL10 = ctrl.Antecedent(np.arange(IL10_0, IL10_end, .05), 'IL10')
        #// define memberships
        # TNFa['L'] = fuzz.trimf(TNFa.universe, [TNFa_0, TNFa_0, TNFa_1])
        # TNFa['M'] = fuzz.trimf(TNFa.universe, [TNFa_0, TNFa_1, TNFa_end])
        # TNFa['H'] = fuzz.trimf(TNFa.universe, [TNFa_1, TNFa_end, TNFa_end])
        IL10['N'] = fuzz.trimf(IL10.universe, [IL10_0, IL10_0,self.params['IL10'][1]])
        IL10['L'] = fuzz.trimf(IL10.universe, [IL10_0, self.params['IL10'][1], self.params['IL10'][2]])
        IL10['M'] = fuzz.trimf(IL10.universe, [self.params['IL10'][1], self.params['IL10'][2],self.params['IL10'][3]])
        IL10['H'] = fuzz.trapmf(IL10.universe, [self.params['IL10'][2],self.params['IL10'][3], IL10_end, IL10_end])
        #// store
        # self.antecedents.append(TNFa)
        self.antecedents.append(IL10)
#         IL10.view()
#         TNFa.view()
        #// define consequents
        range_value = np.arange(0, 1, .01)
        early_diff = ctrl.Consequent(range_value, 'early_diff')
        late_diff = ctrl.Consequent(range_value, 'late_diff')
        
        #// define membership functions
        sigma = .05
        early_diff_intervals = [0,self.params['early_diff_m'], 1]
        early_diff['L']=fuzz.gaussmf(range_value, early_diff_intervals[0], sigma)
        early_diff['M']=fuzz.gaussmf(range_value, early_diff_intervals[1], sigma)
        early_diff['H']=fuzz.gaussmf(range_value, early_diff_intervals[2], sigma)
        
        late_diff_intervals = [0,self.params['late_diff_m'], 1]
        late_diff['L']=fuzz.gaussmf(range_value, late_diff_intervals[0], sigma)
        late_diff['M']=fuzz.gaussmf(range_value, late_diff_intervals[1], sigma)
        late_diff['H']=fuzz.gaussmf(range_value, late_diff_intervals[2], sigma)

        #// Store
        self.consequents = [early_diff,late_diff]
        #// rules
        early_diff_rules = [
            ctrl.Rule(IL10['L'] , early_diff['H']),
            ctrl.Rule(IL10['N'] | IL10['M'] , early_diff['M']),
            ctrl.Rule(IL10['H'] , early_diff['L'])
        ]
        
        late_diff_rules = [
        ctrl.Rule(IL10['L'] , late_diff['H']),
        ctrl.Rule(IL10['N'] | IL10['M'] , late_diff['M']),
        ctrl.Rule(IL10['H'] , late_diff['L'])
        ]
        
        rules = early_diff_rules + late_diff_rules
        self.controler = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules))
        
    def forward(self,inputs):
        for key,value in inputs.items():
            self.controler.input[key] = value
        self.controler.compute()
        # for item in self.consequents:
        #     item.view(sim=self.controler)
        outputs = self.controler.output
        # outputs['IL6'] *= (self.params['IL6'][-1]-self.params['IL6'][0])+self.params['IL6'][0]
        # outputs['PGE2'] *= (self.params['PGE2'][-1]-self.params['PGE2'][0])+self.params['PGE2'][0]
        return outputs
        

        
class MSC_model:
    def __init__(self,free_params_keys,free_params_values,measurement_scheme,inputs):
        self.inputs = inputs
        self.measurement_scheme=measurement_scheme
        #// assmeble the key and values for the free params
        free_params = {}
        for key,value in  zip(free_params_keys,free_params_values):
            free_params[key] = value

        self.params = {
            'IL10': [0,free_params['IL10_L'],(free_params['IL10_L']+free_params['IL10_H'])/2,free_params['IL10_H'],100]
        }
        self.params = {**self.params, **free_params}
        self.data = {'IL6':[0],'PGE2':[0]}
        self.fuzzy_model = Fuzzy_MSC(self.params)
        self.osteogenic_model = None
    def osteogenesis(self,fuzzy_results):
        ALP_checkpoint = self.measurement_scheme['ALP']*24 #time in hours
        ARS_checkpoint = self.measurement_scheme['ARS']*24
        k0 = 1/(self.params['diff_time']*24) # base diff rate in hours
        f_early = fuzzy_results['early_diff'] 
        f_late = fuzzy_results['late_diff'] 
        maturity_t = self.params['maturity_t']*self.params['diff_time']*24 # because given maturity_t in the parameters is between 0 and 1
        k_eraly = f_early*k0
        k_late = f_late*k0
        def maturity_calculator(timepoint):
            if timepoint < maturity_t:
                maturity = k_eraly*timepoint # no for loop
                return maturity
            else:
                maturity_0 = k_eraly*maturity_t
                maturity_1 = k_late*(timepoint-maturity_t)
                return maturity_0+maturity_1
#         print('f_early {} f_late {} k0 {}'.format(f_early,f_late,k0))
        ALP_sim = maturity_calculator(ALP_checkpoint)*self.params['ALP_maturity']
        ARS_sim = maturity_calculator(ARS_checkpoint)*self.params['ARS_maturity']
        return {'ALP':ALP_sim, 'ARS':ARS_sim}        
    def run(self):
        #// get the coeff from fuzzy model
        fuzzy_results = self.fuzzy_model.forward(self.inputs)
        fuzzy_results = self.fuzzy_model.forward(self.inputs)
        results = self.osteogenesis(fuzzy_results)
        print(results)
#         f_early_diff = fuzzy_results['early_diff']
#         f_late_diff = fuzzy_results['late_diff']
        #// run the osteogenic model based on the fuzzy parameters
        
#         return {'IL6':results['IL6'][-1],'PGE2':results['PGE2'][-1]}

free_params_values=[np.mean(item) for item in free_params.values()]
# load the samples from the original model
with open('observations.json') as json_file:
    observations = json.load(json_file)
obj = MSC_model(free_params_keys=free_params.keys(),free_params_values=free_params_values,
                measurement_scheme=observations['Chen_2018']['measurement_scheme'],
               inputs=observations['Chen_2018']['ctr']['inputs'])
obj.run()


        
        

f_early 0.49999999999999967 f_late 0.49999999999999967 k0 0.001851851851851852
{'ALP': 0.13333333333333325, 'ARS': 0.39999999999999974}
